# PyTorch mutual information neural estimation tests

Trivial tests with multivariate Gaussian and uniform distribution

In [ ]:
import sys
sys.path.append("../python")

In [ ]:
import numpy as np

In [ ]:
import torch
import torchkld
import mutinfo

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
#device = "cpu"
print("Device: " + device)
print(f"Devices count: {torch.cuda.device_count()}")

In [ ]:
from tqdm import tqdm, trange

In [ ]:
from mutinfo.distributions.base import *
from mutinfo.distributions.tools import mapped_multi_rv_frozen

In [ ]:
from misc.modules import *
from misc.utils import *
from misc.plots import *

## Dataset

Experimental setup

In [ ]:
mutual_information = 10.0

dataset_type = "CorrelatedNormal"
assert dataset_type in ["CorrelatedNormal", "CorrelatedStudent", "CorrelatedStudent_arcsinh", "CorrelatedUniform", "SmoothedUniform", "UniformlyQuantized"]
degrees_of_freedom = 2 # For Student's distribution

dimension = 1

In [ ]:
randomize_interactions = True
shuffle_interactions = True

if dataset_type == "CorrelatedNormal":
    random_variable = CorrelatedNormal(mutual_information, dimension, dimension, randomize_interactions=randomize_interactions, shuffle_interactions=shuffle_interactions)

elif dataset_type in ["CorrelatedStudent", "CorrelatedStudent_arcsinh"]:
    random_variable = CorrelatedStudent(
        mutual_information, dimension, dimension, degrees_of_freedom, randomize_interactions=randomize_interactions, shuffle_interactions=shuffle_interactions
    )

    if dataset_type == "CorrelatedStudent_arcsinh":
        random_variable = mapped_multi_rv_frozen(random_variable, lambda x, y: (np.arcsinh(x), np.arcsinh(y)), lambda x, y: (np.sinh(x), np.sinh(y)))

    dataset_type += f"_dof_{degrees_of_freedom}"
    
elif dataset_type == "CorrelatedUniform":
    random_variable = CorrelatedUniform(mutual_information, dimension, dimension, randomize_interactions=randomize_interactions, shuffle_interactions=shuffle_interactions)

elif dataset_type == "SmoothedUniform":
    random_variable = SmoothedUniform(mutual_information, dimension, dimension, randomize_interactions=randomize_interactions)

elif dataset_type == "UniformlyQuantized":
    from scipy.stats import norm
    
    random_variable = UniformlyQuantized(mutual_information, dimension, norm(loc=0.0, scale=1.0), randomize_interactions=randomize_interactions)

In [ ]:
n_samples = 10*1024

## Estimating MI

Dataset and dataloader

In [ ]:
x, y = random_variable.rvs(n_samples)
train_dataset = torch.utils.data.TensorDataset(
    torch.tensor(x, dtype=torch.float32),
    torch.tensor(y, dtype=torch.float32)
)

x, y = random_variable.rvs(n_samples)
test_dataset = torch.utils.data.TensorDataset(
    torch.tensor(x, dtype=torch.float32),
    torch.tensor(y, dtype=torch.float32)
)

In [ ]:
batch_size = 512

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader  = torch.utils.data.DataLoader(test_dataset,  batch_size=batch_size, shuffle=False)

Model

In [ ]:
inner_dim = 128

model = BasicDenseT(dimension, dimension, inner_dim=inner_dim).to(device)
total_parameters = sum(parameter.numel() for parameter in model.parameters())
print(f"Total parameters: {total_parameters}")

Loss

In [ ]:
# Loss.
biased = False
ema_multiplier = 1.0e-2
marginalize = "permute" # "permute", "product"

losses = {
    "DonskerVaradhan": torchkld.loss.DonskerVaradhanLoss(biased=biased, ema_multiplier=ema_multiplier),
    "NWJ": torchkld.loss.NWJLoss(),
    "Nishiyama": torchkld.loss.NishiyamaLoss(),
    "InfoNCE": torchkld.loss.InfoNCELoss(),
}

loss_name = "DonskerVaradhan"
loss = losses[loss_name]

Optimizer

In [ ]:
learning_rate = 1.0e-3
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

Training

In [ ]:
import matplotlib
from matplotlib import pyplot as plt

In [ ]:
# Number of epochs used to average the estimate.
average_epochs = 200

In [ ]:
from collections import defaultdict
from IPython.display import clear_output
from tqdm import trange

n_epochs = 2000

history = defaultdict(list)
for epoch in trange(1, n_epochs + 1, mininterval=1):    
    # Training.
    for index, batch in enumerate(train_dataloader):
        x, y = batch
        batch_size = x.shape[0]
        
        optimizer.zero_grad()
        
        T_joined   = model(x.to(device), y.to(device))
        T_marginal = model(x.to(device), y.to(device), marginalize=marginalize)
        _loss = loss(T_joined, T_marginal)
        _loss.backward()
        
        optimizer.step()
        
    history["train_mutual_information"].append(model.get_mutual_information(train_dataloader, loss, device, marginalize=marginalize))
    history["test_mutual_information"].append(model.get_mutual_information(test_dataloader, loss, device, marginalize=marginalize))

    if epoch % 50 == 0:
        clear_output(wait=True)
        plot_estimated_MI_trainig(mutual_information, np.arange(1, epoch+1), history["train_mutual_information"])
        plot_estimated_MI_trainig(mutual_information, np.arange(1, epoch+1), history["test_mutual_information"])
        print(f"Current estimate: {history['test_mutual_information'][-1]:.2f}")
        print(f"Running median: {np.median(history['test_mutual_information'][-average_epochs:]):.2f}")

### Saving the results

In [ ]:
parameters = {
    # Dataset.
    "dataset_type": dataset_type,
    "mutual_information": mutual_information,

    # Model.
    "inner_dim": inner_dim,

    # Loss.
    "loss_name": loss_name,
    "biased": biased,
    "ema_multiplier": ema_multiplier,

    # Training.
    "n_samples": n_samples,
    "batch_size": batch_size,
    "n_epochs": n_epochs,
    "learning_rate": learning_rate,

    # Saving the results.
    "average_epochs": average_epochs,
}

In [ ]:
from datetime import datetime

experiment_name = f"{dataset_type}_{dimension}_{mutual_information:.1f}_{n_samples}__{datetime.now().strftime('%d-%b-%Y_%H:%M:%S')}"
print(experiment_name)

In [ ]:
import os
from pathlib import Path

data_path = Path(os.path.abspath(os.path.join(os.path.abspath(os.getcwd()), "../../data/synthetic")))
experiment_path = data_path / f"{dataset_type}" / loss_name / experiment_name

In [ ]:
save_results(history, parameters, experiment_path, average_epochs=average_epochs)